## Data Wrangling Project

In this project, I imported map data in XMl format to a SQL database for further investigation. In order to accomplish this, the data needed to be cleaned and standardized before being exported to csv files to be uploaded to the database.

I chose to work with a square area encompassing the city of San Jose, CA and some of its surrounding municipalities. San Jose is my hometown and an area near and dear to my heart, so exploring this OpenStreetMap.org map data was of personal interest. The square area dataset I utilized was pre-compiled by mapzen:

https://www.openstreetmap.org/relation/112143

https://mapzen.com/data/metro-extracts/metro/san-jose_california/

A visualization of the area used can be seen in the image below.

The goal was to clean the data and then import them to a SQL database, enabling functionality with which to extract various statistics about the dataset. The source data were relatively clean, but as is the case with any set of human-compiled data, there were some errors.

## Challenges to Data Wrangling

\- There was a fair amount of non-standardization among the street addresses. For example, "Ave" and "Avenue" were both used, when, ideally, the same notation would be used to describe the same type of street. In order to correct this, I used the following function to convert both common stand-ins (such as "St" and "St." for "Streets") and more specific ones to this dataset (such as correcting "Boulvevard" to "Boulevard"):

def update_name(name, mapping):
    
    names = name.split(' ')
    if names[-1] in mapping:    #check if the last word has a corresponding update
        names[-1] = mapping[names[-1]]
        return " ".join(names)
    else:
        return name

\- One street address in particular was labeled "Hwy 17 PM 7.1" and had associated latitude and longitude coordinates corresponding to a location on Loma Alta Avenue in Los Gatos, CA (https://www.google.com/maps/place/37°13'12.0"N+121°58'12.0"W/@37.2212714,-121.978025,15.44z). This is near Highway 17, leading me to believe there is some connection, but as this node is very clearly not on Highway 17, this was likely tagged incorrectly and the problem is easily remedied by removing this tag.

\- There were two tags (one node tag and one way tag) with a "k"-attribute labelled "old_amenity", whereas the standard used in the rest of the data was "amenity". Some "k"-attributes were listed as "abandoned:amenity" or similar, so we will implement a function to change "old_amenity" to "old:amenity" in order to preserve the data while correctly returning "amenity" as the key when implementing the code to convert to csv for importing to SQL.

def check_k(name):
    
    if name == "old_amenity":
        return "old:amenity"
    else:
        return name

## Data Exploration

Filesizes:

raw data (san-jose.osm): 271 MB

database (san-jose.db): 157 MB


Once the data were uploaded to a SQL database, investigation of the dataset could be achieved through various queries.


### Query for finding number of nodes

SELECT COUNT(\*) FROM nodes;

1285096


### Number of ways

SELECT COUNT(\*) FROM ways;

170893

### Unique users listed among nodes and ways entries

SELECT COUNT(DISTINCT query.uid)          
    FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) as query;

1246

This results in an average of approximately 1168 entries per user! What a helpful community.

### Keys in node_tags

We can also investigate what the most populous types of node_tags are.

* highway, 12212
* housenumber, 6746
* street, 6523
* name, 5166
* amenity, 3749
* city, 3309
* postcode, 2814
* natural, 2156
* crossing, 2052
* source, 1960

### Wikipedia keys!

One key of particular interest to me is the "wikipedia" key; how many entries from nodes_tags and ways_tags correspond to this key?

SELECT COUNT(query.key)          
FROM (SELECT key FROM nodes_tags UNION ALL SELECT key FROM ways_tags) as query
WHERE query.key="wikipedia";

137

I wondered what Wikipedia articles were being linked to so I examined 10 "wikipedia" tags from each nodes_tags and ways_tags.

SELECT value FROM nodes_tags WHERE key="wikipedia" LIMIT 10;
* "en:Santa Clara, California"
* "en:Berryessa, San Jose, California"
* "en:Willow Glen, San Jose, California"
* "en:Morgan Hill, California"
* "en:Coyote, California"
* "en:Sunnyvale, California"
* "en:Buena Vista, Santa Clara County, California"
* "en:Alum Rock, California"
* "en:Moffett Park (VTA)"
* "en:Borregas (VTA)"

Node tags with the "wikipedia" key link predominantly to Wikipedia articles on cities or on train stations (VTA is a type of light-rail train that runs in this area).

SELECT value FROM ways_tags WHERE key="wikipedia" LIMIT 10;
* "en:San Jose Convention Center"
* "en:SAP Center at San Jose"
* "en:Bellarmine College Preparatory"
* "en:Plaza de C├⌐sar Ch├ívez"
* "en:Dr. Martin Luther King, Jr. Library"
* "en:San Jose High School"
* "en:City National Civic"
* "en:San Jose Municipal Stadium"
* "en:Shark Ice"
* "en:History Park at Kelley Park"

Way tags with the "wikipedia" key link to locations of public interest, such as the SAP Center (go Sharks!) and local high schools.

### What kind of amenities are in the area?

* restaurant, 766
* fast_food, 369
* cafe, 224
* place_of_worship, 186
* bicycle_parking, 175
* bench, 174
* school, 151
* toilets, 148
* fuel, 123
* bank, 116

Let's take a closer look at that last entry; what bank has the most branches in our area encompassing San Jose?

* Chase, 17
* Bank of America, 15
* Wells Fargo, 14
* Bank of the West, 6
* US Bank, 6
* Citibank, 5
* Provident Credit Union, 4
* Star One Credit Union, 3
* City Bank, 2
* Comerica Bank, 2

Chase, BoA, and Wells Fargo customers appear to be in good hands in this area! Or, at the very least, a large number of hands. Similarly, we can examine the most popular coffee chain in the area.

* Starbucks, 71
* Subway, 6
* Peet's Coffee & Tea, 4
* Bagel Street Cafe, 3
* Jamba Juice, 3
* Peet's Coffee, 3
* Philz Coffee, 3
* Starbucks Coffee, 3
* Peets Coffee & Tea, 2
* 88 Keys Cafe, 1

Interestingly, if we look for all Subway restaurants

SELECT COUNT(\*) FROM nodes_tags WHERE value="Subway";

we find that there are 50 locations and not all of them are tagged as cafes.

## Suggestions for improvement

This dataset provides a good look into the San Jose metropolitan area. However, the dataset itself is very clearly incomplete, as evidenced by the very last exploratory look where only 6 of 50 Subway locations are tagged as cafes. Improved and consistent tagging of locations such as these would allow for more accurate exploration of the data; however, implementing this on a programmatic basis would be difficult as individual exceptions would likely have to be programmed for each case such as creating a corresponding "cafe" tag for all Subway locations (or, alternatively, removing the tag). While that would be simple enouogh for Subway, individual chains (cafe, fast food, restaurant, bank, or otherwise) would need their own self-consistency checks.